Contributors:
Peter Harmer,
Girish Dodda


In [14]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import re
from google.colab import drive

In [2]:
#Initializing
gpu_available = torch.cuda.is_available()
if gpu_available:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [15]:
#Data Load
 #Change as you need to load your own version of the file
drive.mount('/content/drive')
# data = pd.read_csv('drive/My Drive/CS_539/bbc-news-data.csv', delimiter='\t')
data = pd.read_csv('drive/My Drive/COMP SCI 539/bbc-news-data.csv', delimiter='\t')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data.head(5)

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [16]:
#Data Preprocessing

# Find and remove nulls
print(data.isnull().sum())

# Data to lowercase
data["title"] = data["title"].str.lower()
data["content"] = data["content"].str.lower()
# Remove and replace contractions
    #Find more contraction in text and add
contraction_dict = {"can't":"cannot","didn't":"did not","aren't":"are not","she'd":"she would","he'd":"he would","they'd":"they would","they've":"they have",
"shouldn't":"should not","shouldn't've":"should not have","she'll":"she will","he'll":"he will","they'll":"they will"
}

category    0
filename    0
title       0
content     0
dtype: int64


In [6]:
data.head(5)

,category,filename,title,content
0,business,001.txt,ad sales boost time warner profit,quarterly profits at us media giant timewarne...
1,business,002.txt,dollar gains on greenspan speech,the dollar has hit its highest level against ...
2,business,003.txt,yukos unit buyer faces loan claim,the owners of embattled russian oil giant yuk...
3,business,004.txt,high fuel prices hit ba's profits,british airways has blamed high fuel prices f...
4,business,005.txt,pernod takeover talk lifts domecq,shares in uk drinks and food firm allied dome...


In [17]:
#Corpus Creation

def corpus_processing(pandas_dataset, column):
  # Processes the input pandas string columns into a corpus
    # NEEDS: A line to replace contractions
  corpus = [line.strip() for line in pandas_dataset[column] if line.strip()]
  # New Line Here
  corpus = [re.sub('[^A-Za-z0-9]+', ' ', line).lower() for line in corpus]
  corpus = [re.sub(' +', ' ', line) for line in corpus]
  corpus = [word for line in corpus for word in line.split()]
  #corpus =
  return corpus


# Testing on Title column
corpus = corpus_processing(data,'title')
# corpus

vocab_size = 5000
tkn_counter = Counter([word for word in corpus])
vocab = {word: idx for idx, (word, _) in enumerate(tkn_counter.most_common(vocab_size))}
vocab["/UNK"] = len(vocab)

In [8]:
#class DatasetCreation(Dataset):
  #def __init__(self, corpus, vocab, sequence_len=10):

In [9]:
class Encoder(nn.Module):
  def __init__(self, input_size, output_size, hid_dim, emb_dim, n_layers, dropout):
    super().__init__()

    self.hid_dim = hid_dim
    self.n_layers = n_layers

    ###Embedding?


    ### RNN Layer

    self.dropout = nn.Dropout(dropout)

  def forward(self, input):
    i = input

class Decoder(nn.Module):
  def __init__(self, input_size, output_size, hid_dim, emb_dim, n_layers, dropout):
    super().__init__()


    self.out_size = output_size
    self.emb_dim = emb_dim
    self.hid_dim = hid_dim
    self.n_layers = n_layers

    ###Embedding?


    ### RNN Layer

    self.dropout = nn.Dropout(dropout)



In [10]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout_prob):
        super(LSTMModel, self).__init__()

        # Initialize the LSTM
        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=num_layers,
                            dropout=dropout_prob,
                            batch_first=True)

        # Initialize the output layer
        self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Forward pass through LSTM layer
        # x: (batch_size, seq_length, input_size)
        lstm_out, (h_n, c_n) = self.lstm(x)

        # Only take the output from the final timetep
        # Can pass on the h_n or lstm_out to the next layer
        final_state = h_n[-1]

        # Forward pass through the linear layer
        out = self.linear(final_state)

        return out

In [11]:
#Hyperparameters
dropout = 0.1
hidden_dim = 512
lr= 0.01
n_epochs = 200

In [18]:
#Testing for creation errors
# model = base_rnn(5,10,3,4)
model = LSTMModel(len(vocab), hidden_dim, len(vocab), 3, dropout)
model

LSTMModel(
  (lstm): LSTM(3779, 512, num_layers=3, batch_first=True, dropout=0.1)
  (linear): Linear(in_features=512, out_features=3779, bias=True)
)